In [ ]:
%matplotlib inline
#The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook

import cv2
from matplotlib import pyplot as plt
import numpy as np


def adjust_gamma(image, gamma=1.0):
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
	invGamma = 1.0 / gamma
	table = np.array([((i / 255.0) ** invGamma) * 255
		for i in np.arange(0, 256)]).astype("uint8")
	# apply gamma correction using the lookup table
	return cv2.LUT(image, table)



img = cv2.imread('saved_qr_img_2.png')

img = adjust_gamma(img, gamma=0.3)


# ksize 
ksize = (20, 20) 
  
# Using cv2.blur() method  
img = cv2.blur(img, ksize, cv2.BORDER_DEFAULT)


down_width = 900
down_height = 900
down_points = (down_width, down_height)
img = cv2.resize(img, down_points, interpolation= cv2.INTER_LINEAR)

#img = cv2.bitwise_not(img)

#ret, thresh = cv2.threshold(img, 90,255,cv2.THRESH_BINARY )
#img = cv2.bitwise_not(thresh, thresh)

print(np.amax(img))





cv2.imwrite("processed.png", img)

#qcd = cv2.QRCodeDetector()

#qcd.detectAndDecodeMulti(img)




In [ ]:
from qreader import QReader

for i in range(1, 7):

    img = cv2.imread(f'saved_qr_img_{i}.png')


    #img = cv2.bitwise_not(img)
    qreader = QReader()


    sample_id =qreader.detect_and_decode(image=img)

    print(sample_id)

In [ ]:
from statemachine import StateMachine, State, Event


class TrafficLightMachine(StateMachine):
    "A traffic light machine"
    green = State(name = 'green', initial=True,value='green')
    yellow = State('yellow','yellow')
    red = State('red','red')

    cycle = (
        green.to(yellow)
        | yellow.to(red)
        | red.to(green)
    )
    
    
def mount_ok_callback():
    print("mount ok")
    
def unmount_ok_callback():
    print("unmount ok")
    
def mount_error_callback():
    print("mount error")
    
def unmount_error_callback():
    print("unmount error")
    
callbacks =   [
            ('ok','green',mount_ok_callback),
            ('ok','yellow',unmount_ok_callback),
            ('error','mount',mount_error_callback),
            ('error','unmount',unmount_error_callback)
        ]


def run_OK_callbacks(sm:TrafficLightMachine):   
    for callback_tuple in callbacks:
        match callback_tuple:
            case ('ok',sm.current_state.value, callback):
                callback()
                return
            
    print("no callback found")
                


lights  = TrafficLightMachine()
print(lights.current_state.id)
run_OK_callbacks(lights)

lights.cycle()
print(lights.current_state.id)
run_OK_callbacks(lights)

lights.cycle()
print(lights.current_state.id)
run_OK_callbacks(lights)

In [1]:
import asyncio
from qreader import QReader
import cv2

qreader = QReader()
qreader_inv = QReader()

def decode_img(img):
    
   
    qreader = QReader()
    # detect and decode
    sample_ids = qreader.detect_and_decode(image=img)
    #retval, decoded_info, points, straight_qrcode = self.qcd.detectAndDecodeMulti(img)
    # if there is a QR code
    # print the data

    
    if sample_ids != () and sample_ids != (None,):
        
        #print(f"callback input: {sample_ids[0]}")
        return(sample_ids[0])
    
    #raise Exception('could not detect qrcode')
    return None




async def decode_qr(img):
    
    inv_img = cv2.bitwise_not(img)      

    print('creating task for normal image:')
    task_normal = asyncio.create_task(
        asyncio.to_thread(decode_img,img)
    )
    
    print('creating task for inverted image:')
    task_inverted = asyncio.create_task(
        asyncio.to_thread(decode_img,inv_img)
    )
    
    await asyncio.wait([task_normal,task_inverted], return_when=asyncio.FIRST_COMPLETED)
    
    if task_normal.done() and task_normal.result():
        print(f"callback input: {task_normal.result()}")

    if task_inverted.done() and task_inverted.result():
        print(f"callback input: {task_inverted.result()}")
    
    task_inverted.cancel()
    task_normal.cancel()
    

for i in range(1, 7):
    print(f"Decoding Image: {i}" )
    img = cv2.imread(f'saved_qr_img_{i}.png')

    await decode_qr(img)


Decoding Image: 1
creating task for normal image:
creating task for inverted image:
callback input: HZB-V1.0-1test01
Decoding Image: 2
creating task for normal image:
creating task for inverted image:
callback input: HZB-V0.1-01234
Decoding Image: 3
creating task for normal image:
creating task for inverted image:
callback input: HZB-V1.0-1test03
Decoding Image: 4
creating task for normal image:
creating task for inverted image:
callback input: HZB-V1.0-1test02
Decoding Image: 5
creating task for normal image:
creating task for inverted image:
Decoding Image: 6
creating task for normal image:
creating task for inverted image:
